https://github.com/openai/swarm  
https://www.deepseek.com/  
https://bigmodel.cn/console/overview  

In [1]:
import os
import sys
import warnings; warnings.filterwarnings("ignore")
import json
import requests
import torch as th

from pprint import pp
from dotenv import load_dotenv
from openai import OpenAI
from swarm import (Swarm, Agent)
from IPython.display import (Markdown, display)

In [2]:
if sys.platform == "darwin":
    device = th.device("mps")
else:
    device = th.device("cuda" if th.cuda.is_available() else "cpu")
    
devive_cnt = th.cuda.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(f"torch = {th.__version__}")
print(f"cuda = {th.version.cuda}")

device = cuda; devive_cnt = 1
torch = 2.5.1+cu121
cuda = 12.1


In [3]:
path_project = "C:/my_project/MyGit/Machine-Learning-Column/hugging_face"
path_data = os.path.join(os.path.dirname(path_project), "data")
path_output = os.path.join(os.path.dirname(path_project), "output")

if sys.platform == "darwin":
    path_model = "/Users/lukasi33/project/LLM"
else:
    path_model = "F:/LLM"

## step-1: 载入 API KEY

In [4]:
load_dotenv(dotenv_path="explore.env")
deepseek_key = os.getenv("DEEPSEEK_KEY")
baidu_key = os.getenv("BAIDU_KEY")
zhipu_key = os.getenv("ZHIPU_KEY")

## step-2: 实例化客户端

In [77]:
# 调用模型 API
# client = OpenAI(
#     api_key=deepseek_key,
#     base_url="https://api.deepseek.com",
#     timeout=60
#     )

client = OpenAI(
    api_key=zhipu_key,
    base_url="https://open.bigmodel.cn/api/paas/v4",
    timeout=60
    )

In [24]:
# 本地 Ollama
client = OpenAI(
    base_url="http://127.0.0.1:11434/v1",
    api_key="EMPTY",
    timeout=3000
    )

In [25]:
swarm_client = Swarm(client)

## step-3: Agent 推理

In [7]:
# online
# checkpoint = "deepseek-chat"  # v3
# checkpoint = "deepseek-reasoner"  # r1
# checkpoint = "glm-4-plus"

# offline
# checkpoint = "deepseek-r1:14b"
# checkpoint = "qwen2.5:7b"
# checkpoint = "qwen2.5:14b"
checkpoint = "qwen2.5:32b"
# checkpoint = "glm4"
# checkpoint = "llama3.1:8b"
# checkpoint = "llama3.2:3b"

In [8]:
agent = Agent(
    name="Agent",
    model=checkpoint,
    instructions="You are a helpful assistant on business travel."
)

In [9]:
system_prompt = (
    "你叫小慧助手，是由Lukas开发的差旅智能客服。"
    "你的身份是一名差旅秘书，"
    "你的任务是为用户提供基础对话、差旅知识问答、酒店推荐服务。"
    "当问及你的模型参数时，标准回答是属于公司保密信息，要强调模型设计的高效，能够提供高质量的服务。"
    "You are a helpful assistant on business travel."
)

In [10]:
user_prompt = "你好呀~"

In [11]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [12]:
response = swarm_client.run(
    agent=agent,
    messages=messages
)

In [13]:
print(response.messages[-1]["content"])

你好！有什么可以帮到你的吗？是需要差旅建议、酒店推荐还是想知道关于差旅的小贴士呢？


## step-4: Agent 调用工具

In [15]:
cityName2districtId = {
    "北京": "110100",
    "上海": "310100",
    "广州": "440100",
    "深圳": "440300",
    "南京": "320100",
    "杭州": "330100"
}

In [16]:
def get_weather(cityName):
    districtId = cityName2districtId.get(cityName)
    url = f"https://api.map.baidu.com/weather/v1/?district_id={districtId}&data_type=all&ak={baidu_key}"
    response = requests.get(url)
    data = response.json()
    result = data["result"]
    
    result["location"].pop("country")
    result["location"].pop("province")
    result["location"].pop("name")
    result["location"].pop("id")
    
    result["now"].pop("feels_like")
    result["now"].pop("rh")
    result["now"].pop("uptime")
    result["now"].pop("wind_class")
    result["now"].pop("wind_dir")
    
    for _dict in result["forecasts"]:
        _dict.pop("text_night")
        _dict.pop("wc_night")
        _dict.pop("wd_night")
        _dict.pop("wc_day")
        _dict.pop("wd_day")
    return json.dumps(result)

In [17]:
# test tool
result = get_weather(cityName="南京")
result

'{"location": {"city": "\\u5357\\u4eac\\u5e02"}, "now": {"text": "\\u6674", "temp": 10}, "forecasts": [{"text_day": "\\u6674", "high": 10, "low": 0, "date": "2025-02-04", "week": "\\u661f\\u671f\\u4e8c"}, {"text_day": "\\u591a\\u4e91", "high": 9, "low": -1, "date": "2025-02-05", "week": "\\u661f\\u671f\\u4e09"}, {"text_day": "\\u591a\\u4e91", "high": 8, "low": -1, "date": "2025-02-06", "week": "\\u661f\\u671f\\u56db"}, {"text_day": "\\u591a\\u4e91", "high": 2, "low": -5, "date": "2025-02-07", "week": "\\u661f\\u671f\\u4e94"}, {"text_day": "\\u6674", "high": 0, "low": -5, "date": "2025-02-08", "week": "\\u661f\\u671f\\u516d"}]}'

In [18]:
agent = Agent(
    name="Master Agent",
    model=checkpoint,
    instructions="You are a helpful assistant on business travel.",
    functions=[get_weather]
)

In [19]:
# 模拟场景-1
user_prompt = "帮我查下南京的天气"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [26]:
response = swarm_client.run(
    agent=agent,
    messages=messages
)

In [27]:
print(response.messages[-1]["content"])

南京的天气目前为阴。之后几天预计为多云，最高温度最低于10摄氏度。详细天 气情况如下：

* 2025年2月4日：阴，最高温度10摄氏度，最低温度0摄氏度。
* 2025年2月5日：多云，最高温度9摄氏度，最低温度-1摄氏度。
* 2025年2月6日：多云，最高温度8摄氏度，最低温度-1摄氏度。
* 2025年2月7日：多云，最高温度2摄氏度，最低温度-5摄氏度。
* 2025年2月8日：阴，最高温度0摄氏度，最低温度-5摄氏度。


In [69]:
# 模拟场景-2
departCityName = "南京"
arriveCityName = "深圳"
startDate = "2025-02-02"
endDate = "2025-02-10"
travelPurpose = "深圳基地培训"
query = "我明天要去深圳出差，帮我看下天气是否适合出差"

user_prompt = (
    f"出发城市：{departCityName}\n"
    f"到达城市：{arriveCityName}\n"
    f"出差时间：{startDate}至{endDate}\n"
    f"出差目的：{travelPurpose}\n"
    f"需求备注：{query}"
)

In [70]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [71]:
response = swarm_client.run(
    agent=agent,
    messages=messages
)

In [72]:
print(response.messages[-1]["content"])

根据最新的天气预报，深圳未来几天的天气情况如下：

- 2025年2月2日（星期日）：白天阴，夜间小雨，气温13-22℃，风力3-4级。
- 2025年2月3日（星期一）：白天小雨，夜间多云，气温11-14℃，风力3-4级。
- 2025年2月4日（星期二）：白天多云，夜间多云，气温11-18℃，风力小于3级。
- 2025年2月5日（星期三）：白天多云，夜间多云，气温11-19℃，风力小于3级。
- 2025年2月6日（星期四）：白天多云，夜间小雨，气温13-20℃，风力小于3级。

根据这些信息，虽然有几天下雨，但整体气温适宜，风力也不大，应该不会对您的出差造成太大影响。建议您携带一些雨具以防不时之需，并注意根据天气变化适当调整着装。祝您出差顺利！


## step-5: Multi-Agent

In [159]:
# context_variables
# departCityName = "南京"
# arriveCityName = "深圳"
# startDate = "2025-02-05"
# endDate = "2025-02-10"
# travelPurpose = "深圳基地培训"
# customer_context = (
#     f"出发城市：{departCityName}，"
#     f"到达城市：{arriveCityName}，"
#     f"出差时间：{startDate}至{endDate}，"
#     f"出差目的：{travelPurpose}"
# )

# context_variables = {
#     "customer_context": customer_context
# }

In [14]:
# 日常对话助手
dialog_agent = Agent(
    name="日常对话助手",
    model=checkpoint,
    instructions=(
        "你的任务是与用户进行日常对话，你要始终以专业差旅秘书的身份为用户提供引导和建议。"
        "字数控制在100字以内。"
        "不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
    )
)

def transfer_to_dialog_agent():
    return dialog_agent

In [15]:
# 天气预报助手
# https://lbsyun.baidu.com/faq/api?title=webapi/weather/base
cityName2districtId = {
    "北京": "110100",
    "上海": "310100",
    "广州": "440100",
    "深圳": "440300",
    "南京": "320100",
    "杭州": "330100"
}

def get_weather(cityName):
    """
    cityName: 城市名称，如“北京”、“上海”
    """
    districtId = cityName2districtId.get(cityName)
    url = f"https://api.map.baidu.com/weather/v1/?district_id={districtId}&data_type=all&ak={baidu_key}"
    response = requests.get(url)
    data = response.json()
    result = data["result"]
    
    result["location"].pop("country")
    result["location"].pop("province")
    result["location"].pop("name")
    result["location"].pop("id")
    
    result["now"].pop("feels_like")
    result["now"].pop("rh")
    result["now"].pop("uptime")
    result["now"].pop("wind_class")
    result["now"].pop("wind_dir")
    
    for _dict in result["forecasts"]:
        _dict.pop("text_night")
        _dict.pop("wc_night")
        _dict.pop("wd_night")
        _dict.pop("wc_day")
        _dict.pop("wd_day")
    return json.dumps(result)

# weather_agent = Agent(
#     name="天气预报助手",
#     model=checkpoint,
#     instructions=(
#         "你的任务是调用工具帮用户查询指定城市的天气预报，"
#         "一旦你准备好查询天气时，直接调用工具执行查询。"
#         "不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
#         ),
#     functions=[get_weather]
# )

# def transfer_to_weather_agent():
#     return weather_agent

In [16]:
# 查询地点距离助手
# https://lbsyun.baidu.com/faq/api?title=webapi/place-suggestion-api
# https://lbsyun.baidu.com/faq/api?title=webapi/webservice-direction/dirve
def get_distance(city, from_location, end_location):
    """
    city: 城市名称，如“南京”、“杭州”
    from_location: 出发地点，如“全季酒店”
    end_location: 到达地点，如“南研所”
    """
    url_a = f"https://api.map.baidu.com/place/v2/suggestion?query={from_location}&region={city}&city_limit=true&output=json&ak={baidu_key}"
    response_a = requests.get(url_a)
    data_a = response_a.json()
    origin = (data_a["result"][0]["location"]["lat"], data_a["result"][0]["location"]["lng"])
    
    url_b = f"https://api.map.baidu.com/place/v2/suggestion?query={end_location}&region={city}&city_limit=true&output=json&ak={baidu_key}"
    response_b = requests.get(url_b)
    data_b = response_b.json()
    destination = (data_b["result"][0]["location"]["lat"], data_b["result"][0]["location"]["lng"])
    
    # url = f"https://api.map.baidu.com/direction/v2/driving?origin={origin[0]},{origin[1]}&destination={destination[0]},{destination[1]}&tactics=2&ak={baidu_key}"
    url = f"https://api.map.baidu.com/direction/v2/driving?origin={origin[0]},{origin[1]}&destination={destination[0]},{destination[1]}&ak={baidu_key}"
    response = requests.get(url)
    data = response.json()
    distance = data["result"]["routes"][0]["distance"]
    duration = data["result"]["routes"][0]["duration"]
    result = {"distance": distance, "duration": duration}
    return json.dumps(result)

# distance_agent = Agent(
#     name="查询地点距离助手",
#     model=checkpoint,
#     instructions=(
#         "你的任务是调用工具帮用户查询两个地点之间的驾车距离与用时。"
#         "不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
#         # "距离（distance）的单位是米，转换为公里，"
#         # "用时（duration）的单位是秒，转换为分钟。"
#     ),
#     functions=[get_distance]
# )

# def transfer_to_distance_agent():
#     return distance_agent

In [31]:
get_distance(city="南京", from_location="南研所", end_location="丰盛五季酒店")

'{"distance": 1591, "duration": 326}'

In [17]:
# 周边搜助手
# https://lbsyun.baidu.com/faq/api?title=webapi/guide/webservice-placeapi/circle
def get_surround(city, location, query):
    """
    city: 城市名称，如“南京”、“杭州”
    location: 地点，如“全季酒店”、“坂田基地”
    query: 周边类型，如“美食”、“健身中心”
    """
    url_1 = f"https://api.map.baidu.com/place/v2/suggestion?query={location}&region={city}&city_limit=true&output=json&ak={baidu_key}"
    response_1 = requests.get(url_1)
    data_1 = response_1.json()
    origin = (data_1["result"][0]["location"]["lat"], data_1["result"][0]["location"]["lng"])
    
    url_2 = (
        f"https://api.map.baidu.com/place/v2/search?query={query}&location={origin[0]},{origin[1]}&radius=1000&output=json"
        f"&scope=2&sort_name:distance|sort_rule:1&page_size=10&ak={baidu_key}"
    )
    response_2 = requests.get(url_2)
    data_2 = response_2.json()
    
    results = data_2["results"]
    response = []
    for _dict in results:
        d = {
            "name": _dict.get("name"),
            "address": _dict.get("address"),
            "telephone": _dict.get("telephone"),
            "distance": _dict.get("detail_info").get("distance"),
            "overall_rating": _dict.get("detail_info").get("overall_rating"),
            "shop_hours": _dict.get("detail_info").get("shop_hours")
        }
        response.append(d)
    return json.dumps(response)

In [ ]:
response = get_surround(city="深圳", location="坂田基地", query="美食")
response

In [79]:
# knowledge_agent
# https://blog.csdn.net/walkskyer/article/details/137808429
# def get_knowledge(question):
#     """
#     question: 用户的问题，如“我的住宿标准是多少？”
#     """
#     return "查询知识库回答问题"
    
# knowledge_agent = Agent(
#     name="差旅知识问答智能体（knowledge_agent）",
#     model=checkpoint,
#     instructions=(
#         "你的任务是调用工具帮用户回答差旅业务知识。"
#         "不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
#     ),
#     functions=[get_knowledge]
# )

# def transfer_to_knowledge_agent():
#     return knowledge_agent

In [80]:
# recommend_agent
# def get_recommend_hotel(*agrs, **kwargs):
#     """
#     *agrs, **kwargs
#     """
#     # 调召回接口
#     # 调排序接口
#     response = {}
#     return json.dumps(response)

# recommend_agent = Agent(
#     name="酒店推荐智能体（recommend_agent）",
#     model=checkpoint,
#     instructions=(
#         "你的任务是调用工具帮用户推荐合适的酒店，并给出推荐的理由。"
#         "不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
#     ),
#     functions=[get_recommend_hotel]
# )

# def transfer_to_recommend_agent():
#     return recommend_agent

In [18]:
# triage_agent
# def triage_instructions(context_variables):
#     customer_context = context_variables.get("customer_context")
#     instructions = (
#         "任务详情：\n"
#         "你的任务是对用户的请求进行意图识别，并调用工具将请求转移到正确的意图。"
#         "一旦你准备好将请求转移到正确的意图时，调用工具进行转移。"
#         "你不需要知道具体的细节，只需了解意图的类别，类别是：日常对话、查询天气、查询地标距离。"
#         "当你需要更多信息以识别意图时，直接提出问题，而不需要解释你提问的原因。\n"
#         "注意事项：\n"
#         "1、不要与用户分享你的思维过程\n"
#         "2、不要擅自替用户做出不合理的假设\n"
#         "3、当用户的意图不在预设的范围内时，请你以差旅秘书的身份为用户提供恰当的引导和帮助\n"
#         "用户本次出差的背景信息如下：\n"
#         f"{customer_context}。"
#     )
#     return instructions

instructions = (
    "任务详情：\n"
    "你的任务是对用户的请求进行意图识别，并调用工具将请求转移到正确的意图。"
    "一旦你准备好将请求转移到正确的意图时，调用工具进行转移。"
    "你不需要知道具体的细节，只需了解意图的类别，类别是：日常对话、查询天气预报、查询地点间距离、查询周边信息。"
    "当你需要更多信息以识别意图时，直接提出问题，而不需要解释你提问的原因。\n"
    "注意事项：\n"
    "1、不要与用户分享你的思维过程\n"
    "2、不要擅自替用户做出不合理的假设\n"
    "3、当用户的意图不在预设的范围内时，请你以差旅秘书的身份为用户提供恰当的引导和帮助"
)

triage_agent = Agent(
    name="意图识别助手",
    model=checkpoint,
    instructions=instructions,
    functions=[transfer_to_dialog_agent, get_weather, get_distance, get_surround],
    parallel_tool_calls=True
)

In [19]:
system_prompt = (
    "你叫小慧助手，是由Lukas开发的差旅智能客服。"
    "你的身份是一名差旅秘书，"
    "你的任务是为用户提供差旅服务，如：查询天气预报、查询地点间距离、查询周边信息。"
    "当问及你的模型参数时，标准回答是属于公司保密信息，要强调模型设计的高效，能够提供高质量的服务。"
    "You are a helpful assistant on business travel."
)

In [20]:
departCityName = "南京"
arriveCityName = "深圳"
startDate = "2025-02-05"
endDate = "2025-02-10"
travelPurpose = "参加公司培训"
user_context = (
    "------\n"
    "出差背景信息供参考：\n"
    f"  出发城市：{departCityName}\n"
    f"  到达城市：{arriveCityName}\n"
    f"  出发日期：{startDate}\n"
    f"  返回日期：{endDate}\n"
    f"  出差目的：{travelPurpose}"
)

# user_prompt = "你好"
# user_prompt = "你是谁呀"
# user_prompt = "你是什么模型？"
# user_prompt = "南京的天气"
user_prompt = "帮我查下深圳的天气"
# user_prompt = "未来天气是否适合出差？"
# user_prompt = "北京的天气"
# user_prompt = "明天"
# user_prompt = "深圳的天气"
# user_prompt = "我的住宿标准是多少？"
# user_prompt = "帮我推荐个酒店"
# user_prompt = "吴彦祖是谁？"
# user_prompt = "我今天心情不好"
# user_prompt = "和女朋友吵架了怎么办"
# user_prompt = "从南京安朴酒店到华为南研所有多远？"
# user_prompt = "从华为南研所到夫子庙有多远？"
# user_prompt = "给我介绍下南京安朴酒店"
# user_prompt = "这个酒店离华为南研所多远？"
# user_prompt = "帮我查下出发当天的天气情况"
# user_prompt = "我回来那天的天气情况"
# user_prompt = "深圳有哪些特产？"
# user_prompt = "从坂田基地到罗湖口岸有多远，要多久？"
# user_prompt = "深圳坂田基地附近有哪些好吃的粤菜？"
# user_prompt = "深圳安托山附近有哪些酒店，给我一些推荐？"
user_prompt += "\n" + user_context

In [21]:
print(user_prompt)

帮我查下深圳的天气
------
出差背景信息供参考：
  出发城市：南京
  到达城市：深圳
  出发日期：2025-02-05
  返回日期：2025-02-10
  出差目的：参加公司培训


In [22]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [ ]:
# response = swarm_client.run(
#     agent=triage_agent,
#     messages=messages,
#     context_variables=context_variables,
#     stream=False
# )

response = swarm_client.run(
    agent=triage_agent,
    messages=messages,
    stream=True
)
for res in response:
    print(res)

{'delim': 'start'}
{'content': '', 'function_call': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'index': 0, 'id': 'call_gguqzflq', 'function': {'arguments': '{"cityName":"深圳"}', 'name': 'get_weather'}, 'type': 'function'}], 'sender': '意图识别助手'}
{'content': '', 'function_call': None, 'refusal': None, 'role': 'assistant', 'tool_calls': None, 'sender': '意图识别助手'}
{'delim': 'end'}
{'delim': 'start'}
{'content': '深圳今天的天气是晴朗，气温为13℃。接下来几天的天气预报如下：\n- 2月4日（星期二）：多云，最高温度17℃，最低温度11℃。\n- 2月5日（星期三）：多云，最高温度20℃，最低温度11℃。\n- 2月6日（星期四）：阴天，最高温度20℃，最低温度13℃。\n- 2月7日（星期五）：小雨，最高温度20℃，最低温度10℃。\n- 2月8日（星期六）：阴天，最高温度16℃，最低温度9℃。\n\n请注意带适当的衣物，并留意当天的天气变化。祝您出差顺利！', 'function_call': None, 'refusal': None, 'role': 'assistant', 'tool_calls': None, 'sender': '意图识别助手'}
{'delim': 'end'}
{'response': Response(messages=[{'content': '', 'sender': '意图识别助手', 'role': 'assistant', 'function_call': None, 'tool_calls': [{'function': {'arguments': '{"cityName":"深圳"}', 'name': 'get_weather'}, 'id': 'call_gguqzflq', 'typ

In [ ]:
print(response.messages[-1]["content"])